## Rover Project Test Notebook
This notebook contains the functions from the lesson and provides the scaffolding you need to test out your mapping methods.  The steps you need to complete in this notebook for the project are the following:

* First just run each of the cells in the notebook, examine the code and the results of each.
* Run the simulator in "Training Mode" and record some data. Note: the simulator may crash if you try to record a large (longer than a few minutes) dataset, but you don't need a ton of data, just some example images to work with.   
* Change the data directory path (2 cells below) to be the directory where you saved data
* Test out the functions provided on your data
* Write new functions (or modify existing ones) to report and map out detections of obstacles and rock samples (yellow rocks)
* Populate the `process_image()` function with the appropriate steps/functions to go from a raw image to a worldmap.
* Run the cell that calls `process_image()` using `moviepy` functions to create video output
* Once you have mapping working, move on to modifying `perception.py` and `decision.py` to allow your rover to navigate and map in autonomous mode!

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

**Run the next cell to get code highlighting in the markdown cells.**

In [ ]:
%%HTML
<style> code {background-color : orange !important;} </style>

In [2]:
%matplotlib qt
#%matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder

## Quick Look at the Data
There's some example data provided in the `test_dataset` folder.  This basic dataset is enough to get you up and running but if you want to hone your methods more carefully you should record some data of your own to sample various scenarios in the simulator.  

Next, read in and display a random image from the `test_dataset` folder

In [3]:
def get_random_img():
    path = '../test_dataset/IMG/*'
    img_list = glob.glob(path)
    idx = np.random.randint(0, len(img_list)-1)
    print('Fetching random image : ', img_list[idx])
    return mpimg.imread(img_list[idx])
    
plt.imshow(get_random_img())


Fetching random image :  ../test_dataset/IMG\robocam_2017_05_02_11_16_32_075.jpg


## Calibration Data
Read in and display example grid and rock sample calibration images.  You'll use the grid for perspective transform and the rock image for creating a new color selection that identifies these samples of interest. 

In [4]:
# In the simulator you can toggle on a grid on the ground for calibration
# You can also toggle on the rock samples with the 0 (zero) key.  
# Here's an example of the grid and one of the rocks
example_grid = '../calibration_images_actual/calib1.jpg'
example_rock = '../calibration_images/example_rock1.jpg'
grid_img = mpimg.imread(example_grid)
rock_img = mpimg.imread(example_rock)

fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.imshow(grid_img)
plt.subplot(122)
plt.imshow(rock_img)

## Perspective Transform

Define the perspective transform function from the lesson and test it on an image.

In [14]:
# Define a function to perform a perspective transform
# I've used the example grid image above to choose source points for the
# grid cell in front of the rover (each grid cell is 1 square meter in the sim)
# Define a function to perform a perspective transform
def perspect_transform(img, src, dst):
           
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))# keep same size as input image
    
    return warped


# Define calibration box in source (actual) and destination (desired) coordinates
# These source and destination points are defined to warp the image
# to a grid where each 10x10 pixel square represents 1 square meter
# The destination box will be 2*dst_size on each side
dst_size = 5 
# Set a bottom offset to account for the fact that the bottom of the image 
# is not the position of the rover but a bit in front of it
# this is just a rough guess, feel free to change it!
bottom_offset = 6
image = grid_img
source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
destination = np.float32([[image.shape[1]/2 - dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - 2*dst_size - bottom_offset], 
                  [image.shape[1]/2 - dst_size, image.shape[0] - 2*dst_size - bottom_offset],
                  ])
warped = perspect_transform(grid_img, source, destination)
plt.subplot(121)
plt.imshow(image)
plt.subplot(122)
plt.imshow(warped)
#scipy.misc.imsave('../output/warped_example.jpg', warped)

## Color Thresholding

**HSV**

In [ ]:
# Identify pixels above the threshold
# Threshold of RGB > 160 does a nice job of identifying ground pixels only
def color_thresh(img, rgb_thresh_min, rgb_thresh_max):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    mask = (img[:,:,0] >= rgb_thresh_min[0]) & (img[:,:,1] >= rgb_thresh_min[1]) & (img[:,:,2] >= rgb_thresh_min[2]) \
       & (img[:,:,0] <= rgb_thresh_max[0]) & (img[:,:,1] <= rgb_thresh_max[1]) & (img[:,:,2] <= rgb_thresh_max[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[mask] = 1
    # Return the binary image
    return color_select

def rock_thresh(img):
    return color_thresh(img, rgb_thresh_min=(120,0,0), rgb_thresh_max=(220,185,80))

def navigable_thresh(img, clip_top_ratio = 0.48):
    nav_th_img = color_thresh(img, rgb_thresh_min=(120, 100, 60), rgb_thresh_max=(255,255,255))
    top_half = int(clip_top_ratio * img.shape[0])
    nav_th_img[:top_half,:] = 0 # above the horizon its not navigable
    return nav_th_img

def obstacle_thresh(img):
    return color_thresh(img, rgb_thresh_min=(0, 0, 0), rgb_thresh_max=(110,110,110))


#test_image = get_random_img()
test_image = mpimg.imread("../test_dataset/IMG/robocam_2017_05_02_11_16_29_011.jpg")
#test_image = rock_img

hsv_image = cv2.cvtColor(test_image, cv2.COLOR_RGB2HSV_FULL)
threshed = navigable_thresh(hsv_image)
rock_threshed = rock_thresh(hsv_image)
obstacle_threshed = obstacle_thresh(hsv_image)


plt.subplot(231)
plt.imshow(test_image)
plt.subplot(232)
plt.imshow(hsv_image)
plt.subplot(234)
plt.imshow(threshed, cmap='gray')
plt.subplot(235)
plt.imshow(rock_threshed, cmap='gray')
plt.subplot(236)
plt.imshow(obstacle_threshed, cmap='gray')

# RGB Thresholding

In [29]:
# Identify pixels above the threshold
# Threshold of RGB > 160 does a nice job of identifying ground pixels only
def color_thresh(img, rgb_thresh_min, rgb_thresh_max):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    mask = (img[:,:,0] >= rgb_thresh_min[0]) & (img[:,:,1] >= rgb_thresh_min[1]) & (img[:,:,2] >= rgb_thresh_min[2]) \
       & (img[:,:,0] <= rgb_thresh_max[0]) & (img[:,:,1] <= rgb_thresh_max[1]) & (img[:,:,2] <= rgb_thresh_max[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[mask] = 1
    # Return the binary image
    return color_select

def rock_thresh(img):
    return color_thresh(img, rgb_thresh_min=(120,0,0), rgb_thresh_max=(220,185,80))

def navigable_thresh(img, clip_top_ratio = 0.48):
    nav_th_img = color_thresh(img, rgb_thresh_min=(130, 130, 70), rgb_thresh_max=(255,255,255))
    top_half = int(clip_top_ratio * img.shape[0])
    nav_th_img[:top_half,:] = 0 # above the horizon its not navigable
    return nav_th_img

def obstacle_thresh(img):
    return color_thresh(img, rgb_thresh_min=(0, 0, 0), rgb_thresh_max=(110,110,120))


test_image = get_random_img()
#test_image = mpimg.imread("../test_dataset/IMG/robocam_2017_05_02_11_16_29_011.jpg")
#est_image = mpimg.imread("")
#test_image = mpimg.imread("../calibration_images/example_grid1")
#test_image = mpimg.imread("C:/Users/sahas/Desktop/Untitled3.jpg")


#test_image = rock_img

threshed = navigable_thresh(test_image)
rock_threshed = rock_thresh(test_image)
obstacle_threshed = obstacle_thresh(test_image)
warped_nav = perspect_transform(test_image, source, destination)
warped__nav_threshed = perspect_transform(threshed, source, destination)

fig = plt.figure(figsize=(12,6))
plt.subplot(321)
plt.imshow(test_image)
plt.subplot(322)
plt.imshow(threshed, cmap='gray')
plt.subplot(323)
plt.imshow(rock_threshed, cmap='gray')
plt.subplot(324)
plt.imshow(obstacle_threshed, cmap='gray')
plt.subplot(325)
plt.imshow(warped_nav, cmap='gray')
plt.subplot(326)
warped_obs = perspect_transform(warped__nav_threshed, source, destination)
plt.imshow(warped_obs, cmap='gray')
#scipy.misc.imsave('../output/warped_threshed.jpg', threshed*255)



Fetching random image :  ../test_dataset/IMG\robocam_2017_05_02_11_16_22_455.jpg


## Coordinate Transformations
Define the functions used to do coordinate transforms and apply them to an image.

In [ ]:
def rover_coords(binary_img):
    # Identify nonzero pixels
    ypos, xpos = binary_img.nonzero()
    # Calculate pixel positions with reference to the rover position being at the 
    # center bottom of the image.  
    x_pixel = np.absolute(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[0]).astype(np.float)
    return x_pixel, y_pixel

# Define a function to convert to radial coords in rover space
def to_polar_coords(x_pixel, y_pixel):
    # Convert (x_pixel, y_pixel) to (distance, angle) 
    # in polar coordinates in rover space
    # Calculate distance to each pixel
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    # Calculate angle away from vertical for each pixel
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

def rotate_pix(xpix, ypix, yaw):
    # Convert yaw to radians
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated

# Define a function to perform a translation
def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated

# Define a function to apply rotation and translation (and clipping)
# Once you define the two functions above this function should work
def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    # Apply rotation
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    # Apply translation
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    # Perform rotation, translation and clipping all at once
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    # Return the result
    return x_pix_world, y_pix_world

# Grab another random image
idx = np.random.randint(0, len(img_list)-1)
image = mpimg.imread(img_list[idx])
warped = perspect_transform(image, source, destination)
threshed = navigable_thresh(warped)

# Calculate pixel values in rover-centric coords and distance/angle to all pixels
xpix, ypix = rover_coords(threshed)
dist, angles = to_polar_coords(xpix, ypix)
mean_dir = np.mean(angles)

# Do some plotting
fig = plt.figure(figsize=(12,9))
plt.subplot(221)
plt.imshow(image)
plt.subplot(222)
plt.imshow(warped)
plt.subplot(223)
plt.imshow(threshed, cmap='gray')
plt.subplot(224)
plt.plot(xpix, ypix, '.')
plt.ylim(-160, 160)
plt.xlim(0, 160)
arrow_length = 100
x_arrow = arrow_length * np.cos(mean_dir)
y_arrow = arrow_length * np.sin(mean_dir)
plt.arrow(0, 0, x_arrow, y_arrow, color='red', zorder=2, head_width=10, width=2)



## Read in saved data and ground truth map of the world
The next cell is all setup to read your saved data into a `pandas` dataframe.  Here you'll also read in a "ground truth" map of the world, where white pixels (pixel value = 1) represent navigable terrain.  

After that, we'll define a class to store telemetry data and pathnames to images.  When you instantiate this class (`data = Databucket()`) you'll have a global variable called `data` that you can refer to for telemetry and map data within the `process_image()` function in the following cell.  


In [ ]:
# Import pandas and read in csv file as a dataframe
import pandas as pd
# Change this path to your data directory
df = pd.read_csv('../test_dataset2/robot_log.csv')
csv_img_list = df["Path"].tolist() # Create list of image pathnames
# Read in ground truth map and create a 3-channel image with it
ground_truth = mpimg.imread('../calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

# Creating a class to be the data container
# Will read in saved data from csv file and populate this object
# Worldmap is instantiated as 200 x 200 grids corresponding 
# to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels)
# This encompasses the full range of output position values in x and y from the sim
class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = -1 # This will be a running index, setting to -1 is a hack
                        # because moviepy (below) seems to run one extra iteration
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

# Instantiate a Databucket().. this will be a global variable/object
# that you can refer to in the process_image() function below
data = Databucket()


## Write a function to process stored images

Modify the `process_image()` function below by adding in the perception step processes (functions defined above) to perform image analysis and mapping.  The following cell is all set up to use this `process_image()` function in conjunction with the `moviepy` video processing package to create a video from the images you saved taking data in the simulator.  

In short, you will be passing individual images into `process_image()` and building up an image called `output_image` that will be stored as one frame of video.  You can make a mosaic of the various steps of your analysis process and add text as you like (example provided below).  



To start with, you can simply run the next three cells to see what happens, but then go ahead and modify them such that the output video demonstrates your mapping process.  Feel free to get creative!

In [11]:

# Define a function to pass stored images to
# reading rover position and yaw angle from csv file
# This function will be used by moviepy to create an output video
def process_image(img):
    # Example of how to use the Databucket() object defined above
    # to print the current x, y and yaw values 
    # print(data.xpos[data.count], data.ypos[data.count], data.yaw[data.count])

    # TODO: 
    # 1) Define constants
    
    dst_size = 5 
    bottom_offset = 6
    source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    destination = np.float32([[img.shape[1]/2 - dst_size, img.shape[0] - bottom_offset],
                  [img.shape[1]/2 + dst_size, img.shape[0] - bottom_offset],
                  [img.shape[1]/2 + dst_size, img.shape[0] - 2*dst_size - bottom_offset], 
                  [img.shape[1]/2 - dst_size, img.shape[0] - 2*dst_size - bottom_offset],
                  ])
    world_map_size = 200
    map_scale = 10
    
    # 2) Apply perspective transform
    transformed_image = perspect_transform(img, source, destination)
    # 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
    navigable_thresh_image = navigable_thresh(transformed_image)
    obstacle_thresh_image = obstacle_thresh(transformed_image)
    rock_thresh_image = rock_thresh(transformed_image)
    # 4) Convert thresholded image pixel values to rover-centric coords & world centric coords
    navigable_x_rover, navigable_y_rover = rover_coords(navigable_thresh_image)
    navigable_x_world, navigable_y_world = pix_to_world(navigable_x_rover, navigable_y_rover, data.xpos[data.count], data.ypos[data.count], data.yaw[data.count], world_map_size, map_scale)
    
    rock_x_rover, rock_y_rover = rover_coords(rock_thresh_image)
    rock_x_world, rock_y_world = pix_to_world(rock_x_rover, rock_y_rover, data.xpos[data.count], data.ypos[data.count], data.yaw[data.count], world_map_size, map_scale)

    obstacle_x_rover, obstacle_y_rover = rover_coords(obstacle_thresh_image)
    obstacle_x_world, obstacle_y_world = pix_to_world(obstacle_x_rover, obstacle_y_rover, data.xpos[data.count], data.ypos[data.count], data.yaw[data.count], world_map_size, map_scale)
    
    # 6) Update worldmap (to be displayed on right side of screen)
    data.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
    data.worldmap[rock_y_world, rock_x_world, 1] += 1
    data.worldmap[navigable_y_world, navigable_x_world, 2] += 1

    # 7) Make a mosaic image, below is some example code
        # First create a blank image (can be whatever shape you like)
    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1]*2, 3))
        # Next you can populate regions of the image with various output
        # Here I'm putting the original image in the upper left hand corner
    output_image[0:img.shape[0], 0:img.shape[1]] = img

        # Let's create more images to add to the mosaic, the navigable thresh image
        # Add it to the upper right hand corner
    thresh_image_3d = np.dstack((obstacle_thresh_image*255, rock_thresh_image*255, navigable_thresh_image*255)).astype(np.float)
    output_image[0:img.shape[0], img.shape[1]:] = thresh_image_3d

        # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.1, 0)
        # Flip map overlay so y-axis points upward and add to output_image 
    output_image[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)


        # Then putting some text over the image
    cv2.putText(output_image,"Saurav's Mars Rover Video!!!", (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    data.count += 1 # Keep track of the index in the Databucket()
    
    return output_image

## Make a video from processed image data
Use the [moviepy](https://zulko.github.io/moviepy/) library to process images and create a video.
  

In [12]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Define pathname to save the output video
output = '../output/test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=60) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)

scipy.misc.imsave('../output/mapped_world.png', data.worldmap)

NameError: name 'Databucket' is not defined

### This next cell should function as an inline video player
If this fails to render the video, try running the following cell (alternative video rendering method).  You can also simply have a look at the saved mp4 in your `/output` folder

In [ ]:

from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Below is an alternative way to create a video in case the above cell did not work.

In [ ]:
import io
import base64
video = io.open(output, 'r+b').read()
encoded_video = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded_video.decode('ascii')))